In [ ]:
import pickle as pkl
import numpy as np
import os

In [ ]:
# solution_folder = '../results/20210806-balSynIn'
wdir = '../results/20210907-ES1500it'

with open(os.path.join(wdir, 'synWeights.pkl'), 'rb') as f:
    synWeights = pkl.load(f)
    
with open(os.path.join(wdir, 'synWeights_final.pkl'), 'rb') as f:
    synWeightsFinal = pkl.load(f)


In [ ]:
import matplotlib.pyplot as plt

t = 0
for n1, n1conns in sorted(list(synWeights.items()), key=lambda x:x[0]):
    plt.figure(figsize=(10,10))
    for n2, w in n1conns.items():
        plt.plot([x for x,y in w], [y for x,y in w])
    plt.title('STDP weights of neuron{} with other neurons over time'.format(n1))
    plt.legend([str(n2) for n2 in n1conns.keys()])
    plt.show()
    t += len(n1conns)
    
print(t)

In [ ]:
import matplotlib.pyplot as plt

all_outs = []
for n1, n1conns in synWeights.items():
    for n2, w in n1conns.items():
        all_outs.append(n2)
        
all_outs = sorted(list(set(all_outs)))
    
t = 0
for n2 in all_outs:
    plt.figure(figsize=(10,10))
    legs = []
    for n1, n1conns in synWeights.items():
        for n2c, w in n1conns.items():
            if n2c == n2:
                plt.plot([x for x,y in w], [y for x,y in w])
                legs.append(n1)
    plt.title('STDP INPUT weights of neuron{}'.format(n2))
    plt.legend([str(n1) for n1 in legs])
    plt.show()
    t += len(n1conns)
    
print(t)

In [ ]:
synWeights.keys()

In [ ]:
import json
import math

sim_config = os.path.join(wdir, 'sim.pkl')
with open(sim_config, 'rb') as f:
  sim = pkl.load(f)

dconf_path = os.path.join(wdir, 'backupcfg_sim.json')
with open(dconf_path, 'r') as f:
  dconf = json.load(f)
pop_sizes = dconf['net']['allpops']

separate_movement = True

def _get_pop_name(cellId, sorted_min_ids):
  # This is O(n), could be O(log(n)) with binary search
  return [pop for pop, minId in sorted_min_ids if cellId >= minId][0]


def _extract_sorted_min_ids(sim, dconf, separate_movement):
  pop_sizes = dconf['net']['allpops']
  sorted_min_ids = sorted(list(sim['simData']['dminID'].items()), key=lambda x:x[1], reverse=True)
  if separate_movement:
    for pop, moves in dconf['pop_to_moves'].items():
      pop_size = pop_sizes[pop]
      move_size = math.floor(pop_size / len(moves))
      smin_dict = dict(sorted_min_ids)
      pop_minId = smin_dict[pop]
      del smin_dict[pop]
      for midx, move in enumerate(moves):
        semi_pop_name = '{}-{}'.format(pop, move)
        smin_dict[semi_pop_name] = pop_minId + midx * move_size
        pop_sizes[semi_pop_name] = move_size
      sorted_min_ids = sorted(list(smin_dict.items()), key=lambda x:x[1], reverse=True)
  return sorted_min_ids

sorted_min_ids = _extract_sorted_min_ids(sim, dconf, separate_movement)

In [ ]:
_get_pop_name(13, sorted_min_ids)

In [ ]:
def group_by_pop(synWeights):
    new_map = {}
    for n1, n1conns in synWeights.items():
        n1pop = _get_pop_name(n1, sorted_min_ids)
#         if n1pop not in new_map:
#             new_map[n1pop] = {}
        for n2, wl in n1conns.items():
            n2pop = _get_pop_name(n2, sorted_min_ids)
#             if n2pop not in new_map[n1pop]:
#                 new_map[n1pop][n2pop] = []
            conn_name = '{}-{}'.format(n1pop, n2pop)
            for idx,(t,w) in enumerate(wl):
                if conn_name not in new_map:
                    new_map[conn_name] = []
                if idx == len(new_map[conn_name]):
                    new_map[conn_name].append([])
                new_map[conn_name][idx].append(w)
    return new_map

popWeights = group_by_pop(synWeights)

In [ ]:
popWeights['ES-EA']

In [ ]:
import matplotlib.pyplot as plt

plt.hist(popWeights['ES-EA'][0], bins=30)

In [ ]:
# last_index = sorted(list(popWeights['ES-EA'].keys()))[-1]
# print(last_index)

plt.hist(popWeights['ES-EA'][-1], bins=30)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
nbins = 50
# color = 'r'

extracted_weights = popWeights['EA-EA']
all_weights = [w for ws in extracted_weights for w in ws]
wmin = np.min(all_weights)
wmax = np.max(all_weights)

for z, weights in enumerate(extracted_weights):
#     ys = np.random.normal(loc=10, scale=10, size=2000)

    hist, bins = np.histogram(weights, bins=nbins, range=(wmin, wmax))
    xs = (bins[:-1] + bins[1:])/2

    ax.plot(xs, hist, zs=z, zdir='y', alpha=0.8)

ax.set_title('EA-EA weight changes over time')
ax.set_xlabel('Weights')
ax.set_ylabel('Time ()')
ax.set_zlabel('Count of neurons')

plt.show()

In [ ]:
popWeights.keys()

In [ ]:
lastt = 0

for neurs, neurvs in zip(*[[k,v] for k,v in synWeights[128].items()]):
    for t in len(neurvs[0]):
        for v in neurvs:
            if v[t] == nan:


In [ ]:
# measure the links of neurons with itself

t = 0
plt.figure(figsize=(4,4))
legend = []
for n1, n1conns in sorted(list(synWeights.items()), key=lambda x:x[0]):
    for n2, w in n1conns.items():
        if n1 == n2:
            print('here')
            plt.plot([x for x,y in w], [y for x,y in w])
            legend.append(n1)
    t += len(n1conns)
    
plt.title('STDP weights of neurons with themselves')
plt.legend(legend)
plt.show()
    
print(t)

In [ ]:
import pandas as pd

updates = {}
LEN_W = None
for n1, n1conns in synWeights.items():
    for n2, w in n1conns.items():
        if not LEN_W:
            LEN_W = len(w)
            updates = dict([(i,[]) for i in range(len(w)-1)])
        assert LEN_W == len(w)
        for i, (x1,y1) in enumerate(w):
            if i == len(w) - 1:
                break
            x2,y2 = w[i+1]
            
            updates[i].append(y2-y1)

df_st = pd.DataFrame(
    [(np.mean(chgs), np.std(chgs), np.amin(chgs), np.amax(chgs)) for i,chgs in updates.items()],
    columns=['mean', 'std', 'min', 'max'])
df_st

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
df_st.describe()

In [ ]:
0.289943 / 1.8

### Look at critic rewards and punishments

In [ ]:
import csv

header = ['time', 'move', 'critic']

rewards = []
with open(os.path.join(solution_folder, 'ActionsRewards.txt')) as f:
    for row in csv.reader(f, delimiter='\t'):
        j = dict([(k, float(v)) for k,v in zip(header, row)])
        rewards.append(j)

In [ ]:
pos_rewards = [row for row in rewards if row['critic'] > 0]
neg_rewards = [row for row in rewards if row['critic'] < 0]

In [ ]:
pos_rewards

In [ ]:
plt.figure(figsize=(20,10))

plt.plot([k['time'] for k in pos_rewards], [k['critic'] for k in pos_rewards])
plt.plot([k['time'] for k in neg_rewards], [k['critic'] for k in neg_rewards])

plt.legend(['reward', 'punishment'])
plt.xlabel('time (ms)')
plt.ylabel('critic')

plt.show()